# M-Shots Learning

In this notebook, we'll explore small prompt engineering techniques and recommendations that will help us elicit responses from the models that are better suited to our needs.

In [1]:
from openai import OpenAI
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

# Formatting the answer with Few Shot Samples.

To obtain the model's response in a specific format, we have various options, but one of the most convenient is to use Few-Shot Samples. This involves presenting the model with pairs of user queries and example responses.

Large models like GPT-3.5 respond well to the examples provided, adapting their response to the specified format.

Depending on the number of examples given, this technique can be referred to as:
* Zero-Shot.
* One-Shot.
* Few-Shots.

With One Shot should be enough, and it is recommended to use a maximum of six shots. It's important to remember that this information is passed in each query and occupies space in the input prompt.



In [53]:
# Function to call the model.
def return_OAIResponse(user_message, context):
    client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)

    newcontext = context.copy()
    newcontext.append({'role':'user', 'content':"question: " + user_message})

    response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=newcontext,
            temperature=1,
        )

    return (response.choices[0].message.content)

In this zero-shots prompt we obtain a correct response, but without formatting, as the model incorporates the information he wants.

In [54]:
#zero-shot
context_user = [
    {'role':'system', 'content':'You are an expert in F1.'}
]
print(return_OAIResponse("Who won the F1 2000?", context_user))

Michael Schumacher won the F1 World Championship in 2000 driving for Scuderia Ferrari.


For a model as large and good as GPT 3.5, a single shot is enough to learn the output format we expect.


In [4]:
#one-shot
context_user = [
    {'role':'system', 'content':
     """You are an expert in F1.

     Who won the 2000 f1 championship?
     Driver: Michael Schumacher.
     Team: Ferrari."""}
]
print(return_OAIResponse("Who won the F1 2011?", context_user))

Driver: Sebastian Vettel.
Team: Red Bull Racing.


Smaller models, or more complicated formats, may require more than one shot. Here a sample with two shots.

In [6]:
#Few shots
context_user = [
    {'role':'system', 'content':
     """You are an expert in F1.

     Who won the 2010 f1 championship?
     Driver: Sebastian Vettel.
     Team: Red Bull Renault.

     Who won the 2009 f1 championship?
     Driver: Jenson Button.
     Team: BrawnGP."""}
]
print(return_OAIResponse("Who won the F1 2006?", context_user))

Driver: Fernando Alonso.
Team: Renault.


In [8]:
print(return_OAIResponse("Who won the F1 2019?", context_user))

Driver: Lewis Hamilton. 
Team: Mercedes.


We've been creating the prompt without using OpenAI's roles, and as we've seen, it worked correctly.

However, the proper way to do this is by using these roles to construct the prompt, making the model's learning process even more effective.

By not feeding it the entire prompt as if they were system commands, we enable the model to learn from a conversation, which is more realistic for it.

In [41]:
#Recomended solution
context_user = [
    {'role':'system', 'content':'You are and expert in f1.\n\n'},
    {'role':'user', 'content':'Who won the 2010 f1 championship?'},
    {'role':'assistant', 'content':"""Driver: Sebastian Vettel. \nTeam: Red Bull. \nPoints: 256. """},
    {'role':'user', 'content':'Who won the 2009 f1 championship?'},
    {'role':'assistant', 'content':"""Driver: Jenson Button. \nTeam: BrawnGP. \nPoints: 95. """},
]

print(return_OAIResponse("Who won the F1 2019?", context_user))

Driver: Lewis Hamilton. 
Team: Mercedes. 
Points: 413.


We could also address it by using a more conventional prompt, describing what we want and how we want the format.

However, it's essential to understand that in this case, the model is following instructions, whereas in the case of use shots, it is learning in real-time during inference.

In [47]:
context_user = [
    {'role':'system', 'content':"""You are and expert in f1.
    You are going to answer the question of the user giving the name of the rider,
    the name of the team and the points of the champion, following the format:
    Drive:
    Team:
    Points: """
    }
]

print(return_OAIResponse("Who won the F1 2019?", context_user))

Driver: Lewis Hamilton
Team: Mercedes
Points: 413


In [49]:
context_user = [
    {'role':'system', 'content':
     """You are classifying .

     Who won the 2010 f1 championship?
     Driver: Sebastian Vettel.
     Team: Red Bull Renault.

     Who won the 2009 f1 championship?
     Driver: Jenson Button.
     Team: BrawnGP."""}
]
print(return_OAIResponse("Who won the F1 2006?", context_user))

Driver: Fernando Alonso.
Team: Renault.


Few Shots for classification.


In [50]:
context_user = [
    {'role':'system', 'content':
     """You are an expert in reviewing product opinions and classifying them as positive or negative.

     It fulfilled its function perfectly, I think the price is fair, I would buy it again.
     Sentiment: Positive

     It didn't work bad, but I wouldn't buy it again, maybe it's a bit expensive for what it does.
     Sentiment: Negative.

     I wouldn't know what to say, my son uses it, but he doesn't love it.
     Sentiment: Neutral
     """}
]
print(return_OAIResponse("I'm not going to return it, but I don't plan to buy it again.", context_user))

Sentiment: Negative


# Exercise
 - Complete the prompts similar to what we did in class. 
     - Try at least 3 versions
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong
 - What did you learn?

In [76]:
def sendRequestLLM(user_message, context):
    client = OpenAI()

    new_context = context.copy()
    new_context.append({"role": "user", "content": user_message })

    res = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=new_context,
        temperature=0.1, # Lower temperature for more deterministic responses
        max_tokens=150, # Limit the response length
    )

    return res.choices[0].message.content

In [84]:
ctx = [
    {"role": "system", "content": "You're an expert in Master Chef Spain. If you are asked for the winner in a given edition, anwer with the name of the winner. If you don't know the answer, respond with 'Sorry, no data available.'"},
    {"role": "user", "content": "Who won 'MasterChef España (formato aficionados)' in 2013?"},
    {"role": "assistant", "content": "Juan Manuel Sánchez."},
    {"role": "user", "content": "Who won 'MasterChef España (formato aficionados)' in 2014?"},
    {"role": "assistant", "content": "Vicky Pulgarín."},
    {"role": "user", "content": "Who won 'MasterChef España (formato aficionados)' in 2015?"},
    {"role": "assistant", "content": "Carlos Maldonado."},
    # editions in the coming future
    {"role": "user", "content": "Who won 'MasterChef España (formato aficionados)' in 2030?"},
    {"role": "assistant", "content": "Sorry, no data available."},
    # editions in the past, before the first edition even took place
    {"role": "user", "content": "Who won 'MasterChef España (formato aficionados)' in 2005?"},
    {"role": "assistant", "content": "Sorry, no data available."},
]


In [88]:
answer = sendRequestLLM("Who won 'MasterChef España (formato aficionados)' in 2013?", ctx)
print(answer)

Juan Manuel Sánchez.


In [89]:
answer = sendRequestLLM("Who won 'MasterChef España (formato aficionados)' in 2014?", ctx)
print(answer)

Vicky Pulgarín.


In [92]:
answer = sendRequestLLM("Who won 'MasterChef España (formato aficionados)' in 2016?", ctx)
print(answer)

Virginia.


In [93]:
answer = sendRequestLLM("Who won 'MasterChef España (formato aficionados)' in 2020?", ctx)
print(answer)

Ana Iglesias.


In [98]:
answer = sendRequestLLM("Who won 'MasterChef España (formato aficionados)' in 1992?", ctx)
print(answer)

Sorry, no data available.


In [101]:
answer = sendRequestLLM("Who won 'MasterChef España (formato aficionados)' in 2024?", ctx)
print(answer)

Sorry, no data available.


In [ ]:
# 
# follow up on previous questions
# 

answer = sendRequestLLM("Who won 'MasterChef España (formato aficionados)' in 2020?", ctx)
print(answer)


answer = sendRequestLLM("Where is she from?", ctx)
print(answer) # asking a follow-up question about the previous answer, will not work as expected because the model does not remember previous interactions (unless you provide the context again)


answer = sendRequestLLM("Where is the person that won 'MasterChef España (formato aficionados)' from?", ctx)
print(answer)

Ana Iglesias.
Sorry, no data available.
The winners of MasterChef España (formato aficionados) come from different regions of Spain.


In [ ]:
# 
# asking specifically for info after the training cutoff date
# (GPT-3.5 Turbo model's knowledge cutoff date is September 2021)
# 

answer = sendRequestLLM("Who won the NBA in 2020?", [])
print(answer) # Los Angeles Lakers --> correct

answer = sendRequestLLM("Who won the NBA in 2021?", [])
print(answer) # Milwaukee Bucks --> correct

answer = sendRequestLLM("Who won the NBA in 2022?", [])
print(answer) # "I am unable to provide real-time information" --> good, at least the model is not hallucinating an answer

answer = sendRequestLLM("Who won the NBA in 2024?", [])
print(answer) # good as well. To get the same format, we could do few-shot.


The Los Angeles Lakers won the NBA championship in 2020.
The Milwaukee Bucks won the NBA championship in 2021.
I am sorry, but I am unable to provide real-time information as I do not have access to the internet. Please check the latest news or sports websites for the most up-to-date information on the winner of the NBA in 2022.
I'm sorry, but I am unable to provide real-time information as I am an AI assistant and do not have access to current data. Please check the latest news or sports websites for the most up-to-date information on the winner of the NBA in 2024.
I'm sorry, but as an AI, I do not have access to real-time information. I recommend checking the latest sports news or the official NBA website to find out who won the NBA in 2042.


In [ ]:
# 
# asking specifically for info after the training cutoff date (second iteration)
# 


ctx = [
    {"role": "system", "content": "You're an expert in Basketball and the NBA. If you're asked for the winner of the NBA, answer only with the name of the winning team. If you can not find the answer, respond with 'Sorry, no data available.'"},
    {"role": "user", "content": "Who won the NBA in 1993?"},
    {"role": "assistant", "content": "Chicago Bulls."},
    {"role": "user", "content": "Who won the NBA in 1994?"},
    {"role": "assistant", "content": "Houston Rockets."},
    {"role": "user", "content": "Who won the NBA in 2042?"},
    {"role": "assistant", "content": "Sorry, no data available."},
]


answer = sendRequestLLM("Who won the NBA in 2020?", ctx)
print(answer) 

answer = sendRequestLLM("Who won the NBA in 2021?", ctx)
print(answer)

answer = sendRequestLLM("Who won the NBA in 2022?", ctx)
print(answer) 

answer = sendRequestLLM("Who won the NBA in 2024?", ctx)
print(answer)


Los Angeles Lakers.
Milwaukee Bucks.
Milwaukee Bucks.
Sorry, no data available.


In [120]:
# 
# beggin for web search
# 


answer = sendRequestLLM("Who won the NBA in 2024?", [])
print(answer) 

answer = sendRequestLLM("I really really need to know who won the NBA in 2024. Can you please do some research, find the answer and tell me who won?", [])
print(answer) 


I'm sorry, but as an AI, I do not have real-time information. Please check the latest news or sports websites for the most up-to-date information on the winner of the NBA in 2024.
I'm sorry, but as an AI, I do not have the capability to browse the internet in real-time. However, you can easily find the answer by searching for "NBA champion 2024" on a search engine or checking the official NBA website for the most recent information.


In [ ]:
#
# Key findings:
#
#
# 1. For accurate results and deterministic responses, using a low temperature is essential. This reduces randomness in the model's responses, making it more reliable for factual queries. In the examples above, where we need factual information, using a temperature of 1 was giving unpredictable results.
#
#
# 2. When dealing with LLM apis, it is important to iterate and refine, as the first attempt may not yield the desired results. In addition to that, it is important to cover the edge cases, as the model may not be able to answer questions about them.
#
#
# 3. OpenAI API is stateless and does not remember previous interactions. It is important to provide context for each request, as they do not retain memory of previous interactions.
#
#
# 4. As of today, most LLM APIs do not support web browsing or real-time data retrieval, so responses are based on the training data available up to a certain date. It's up to the developer to implement web search by combining the LLM API with a web search API.
#    - Note: Perplexity seems to be offering that for Sonar Pro API and I think it's reasonable to expect that from most LLM APIs in the upcoming future (since they're already offering this on their user interface versions).
#    - More info: https://chatgpt.com/share/687e668d-724c-8003-b3bd-81c6ccef9924
#    - Sonar Pro API: https://www.perplexity.ai/hub/blog/introducing-the-sonar-pro-api
#
#